# Markov chains

A Markov chain is a system whose evolution is described by a stoachastic process. If the structure of such process is that dependence of current state with respect to entire past is completely captured by the dependence on the last sample, also known as the markov property, then this process is a markov chain

The markov property is mathematically expressed by the following conditional independence:
(https://en.wikipedia.org/wiki/Conditional_independence):

$$ P(X_{n+1}|X_n, X_{n-1}, ... X_1) = P(X_{n+1}|X_n) $$

That is, the probability of next state within a process is the governed only by the current state.

http://setosa.io/ev/markov-chains/


In [1]:
import os, sys
import itertools
import numpy as np
import pandas as pd
import tensorflow as tf

# Numpy linear algebra package

Example:

Consider a radio station playing 2 genres of songs, which we label as 0 and 1. From time to time, the radio station may switch genre or keep playing songs from the same genre. These transition probabilities can be collected in a matrix (known as the transition matrix). 

The markov chain can be used to answer questions such as:

* What's the probability of current state at any moment? Or likelihood of each genre being played?
    * This is asking for the stationary distribution
* What's the average time for the system to go back to each state? Or how often does it play the same genre again?

To answer these questions, it's necessary to first realize that it's possible that some states could only occur a finite number of times...ie: there may be states that don't recur.

A markov chain is said to be ergodic when we can substitute time averages for ensemble averages:
$$ \lim _ {k \to \infty } {\frac{1}{\frac{1}{k} \sum_{k'=1}^{k}{T_i(k')} }} = \frac {1}{E[T_i(k)]}$$

where $T_i(k)$ is time elapses between (k-1)-th and k-th return to state i.

The **ergodicity theorem** states that

1. If a **stationary distribution** $\pi$ exists for a Markov Chain, it's ergodic.
2. Such **stationary distribution** is independent of initial distribution $\pi_0$ of Markov Chain if it's ergodic.

Therefore, assuming existence of a stationary distribution of the Markov Chain,
the solution can be found using the **power method**:

$$ \pi = \lim_{n \to \infty }{\pi_0 P^{n}}$$

Suppose tha radio station has 1/3 probability of switching genre and 2/3 probability of staying in the same genre. Based on the fact that this is symmetric with respect to each genre, we should intuitively have probability of tuning into each genre equal to 1/2. Now, let's solve using the power method and verify

In [2]:
# transition matrix
P = np.array([[2/3, 1/3], 
              [1/3, 2/3]])

In [3]:
# initial distribution can be anything that sums up to 1
pi0 = np.array([0.5, 0.5])

In [4]:
# compute stationary distribution - power method
np.dot(pi0, np.linalg.matrix_power(P, 50))

array([ 0.5,  0.5])

We see that the probability of state 0 and 1 are both 0.5. But what if probability of switching from 0 to 1 is 1/4 and probability of switching from 1 to 0 is 1/3 so that this radio station baises towards one genre? We can now answer this less trivial question as well with the power method. 

In [5]:
# transition matrix
P = np.array([[3/4, 1/4], 
                [1/3, 2/3]])


In [6]:
# initial distribution can be anything that sums up to 1
pi0 = np.array([0.5, 0.5])

In [7]:
# compute stationary state - power method
np.dot(pi0, np.linalg.matrix_power(P, 50))

array([ 0.57142857,  0.42857143])

We see a new probability of each genre given the changed transition probabilities. Intuitively, we are less likely to switch away from genre 0 to 1 than vice versa and therefore more likely to hear a song from genre 0. Another tool for solving this problem is eigen-decomposition due to Perron-Frobenius theorem.

https://en.wikipedia.org/wiki/Perron–Frobenius_theorem

In [8]:
# some random 5x5 transition matrix
P = np.random.rand(5, 5)
P /= P.sum(axis = 1)[:, np.newaxis] # normalization along axis 1

In [9]:
# compute stationary score - power method
pi0 = np.random.rand(5)
pi0 /= pi0.sum()
a = np.dot(pi0, np.linalg.matrix_power(P, 50))
print(a)

[ 0.24792737  0.23929169  0.17282013  0.2065674   0.13339341]


In [11]:
# compute stationary state - eigen decomposition
L, Q = np.linalg.eig(P.T)

# pick eigenvector whose corresponding eigenvalue is closest to 1
b = Q[:, np.argmin(abs(L - 1.0))].real

# normalize into a probability distribution
b /= b.sum()
print(b)

[ 0.24792737  0.23929169  0.17282013  0.2065674   0.13339341]


In [12]:
np.allclose(a, b)

True

Here's an implementation of the power method with tensorflow. Currently, neither matrix power or eigen-decomposition is supported but we may still want to leverage tensorflow's high scalability to process datasets. This algo will use the divide and conquer strategy to reduce time complexity as well as space complexity for representing such computation in tensorflow. The advantage is that this program will utilize GPU when available and you wouldn't need to change anything to let that happen

In [13]:
# compute stationary state - power method

def mat_power(M, n):
    """ Construct a graph that raises square matrix M to n-th power where n>=1
    This generates a computational graph with space complexity O(log(n)).
    """
    
    assert n >= 1
    
    # trivial cases
    
    if n == 2:
        return tf.matmul(M, M)
    elif n == 1:
        return M
    
    # divide and conquer
    A = mat_power(M, n//2)
    A2 = tf.matmul(A, A)
    
    if n &1: # odd power
        return tf.matmul(A2, M)
    else: # even power
        return A2

In [14]:
def get_stationary_state(P):
    pi0 = tf.constant(np.ones((1, len(P)))/len(P))
    transition_matrix = tf.constant(P)
    stationary_state = tf.squeeze(tf.matmul(pi0, mat_power(transition_matrix, 50)))
    with tf.Session() as sess:
        return sess.run(stationary_state)


In [15]:
a = get_stationary_state(P)
print(a)

[ 0.24792737  0.23929169  0.17282013  0.2065674   0.13339341]


In [16]:
np.allclose(a, b)

True